# Authentication Use Case (1:n Identification)
-----

Here we are going to take a look at the Authentication use case. Here we simply ask a user to snap a selfie the system will then compare the selfie to registered users. The following diagram details the process. 



![Authentication](authentication_diagram.png "Authentication")

1. user initiates an authentication to an application 
2. user snaps a selfie which will be used to compare to the drivers license
3. system makes image quality checks
4. system compares the identification image to the selfie image 
    - if the similarity is above the specified threshold then we say that the faces match 
    - if the similarity is below the specified threshold then  we say that the faces DON'T match
    

In [ ]:
import io
import boto3
import json
from IPython.display import Image as IImage
import pandas as pd

%store -r bucket_name
mySession = boto3.session.Session()
aws_region = mySession.region_name
print("AWS Region: {}".format(aws_region))
print("AWS Bucket: {}".format(bucket_name))

### Setup Clients 
-----
Here we are going to use both S3 and Rekognition apis  

In [ ]:
s3_client  = boto3.client('s3')
rek_client = boto3.client('rekognition')

## Setup Collection and Index  Faces


-----

Here we'll create a collection and index faces from our image mapping file into a panda's dataframe

<div class="alert alert-info"><strong> Next Steps: </strong>
- CreateCollection: creates a searchable index of faces 
- IndexFaces: index a single image into our collection, index multiple faces into our collection 
</div>



In [ ]:
## Name your Collection 
collection_name = 'identity-verification-collection'

try:
    response = rek_client.create_collection(
        CollectionId=collection_name)
except:
    rek_client.delete_collection(
        CollectionId=collection_name)
    response = rek_client.create_collection(
        CollectionId=collection_name)
    
response

## Image map
-----
Here we read an excel file of images to index, the image_map data frame contains the external identifier (reference name) and a reference image as well as a sample selfie image. we'll use the selfie to search the collection. 

In [ ]:
# -- read the image map into a pandas dataframe --
obj = s3_client.get_object(Bucket=bucket_name, Key='IDVImageMapping.xlsx')
image_map = pd.read_excel(io.BytesIO(obj['Body'].read()))
image_map.head()

## Index lots of faces
----

Here we are going to index lots of faces from the image map data frame.

In [ ]:
## Index lots of faces 
dict_of_faces = image_map[["reference_name","reference_image"]].to_dict('records')

for rec in dict_of_faces:
    
    response = rek_client.index_faces(
        CollectionId= collection_name,
        Image={
            'S3Object': {
                'Bucket': bucket_name,
                'Name': rec["reference_image"],
            }
        },
        ExternalImageId=rec['reference_name'],
        DetectionAttributes=[
            'DEFAULT',
        ],
        MaxFaces=1, # maximum faces detected 
        QualityFilter='AUTO' # apply the quality filter. 
        )
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("ImageName: {}, FaceID: {}".format(rec["reference_image"], face_id))
    

print("--- indexing complete ---")

## Step 2. User Snaps a  Selfie   
-----
The image below is a sample selfie which we'll use to search the collection with

In [ ]:
## Image of a Face
selfie_image = "face_3_0.jpeg"
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : selfie_image})))

## Step 3. Image quality checks
----

Here we want to do some basic checks:
1. that we can detect that there is only one face in the selfie 
2. the quality (sharpness and brightness) are sufficient to match with 

Note: we could do several other checks, but we'll see those in module 3.


<div class="alert alert-info"><strong> DetectFaces </strong>
    
The **DetectFaces** operation that looks for key facial features such as eyes, nose, and mouth to detect faces in an input image. Amazon Rekognition Image detects the 100 largest faces in an image.

Here we actually will detect two faces in the driver's license. 

</div>

<div class="alert alert-info"><strong> Note </strong>
    
Take a look at the FaceDetails of each face detected. it will return several helpful details of image quality for each face in the image. 

- BoundingBox
- AgeRange
- Gender
- Landmarks
- Quality
- Pose 

</div>

In [ ]:
# -- selfie check   -- 
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket':bucket_name,
    'Name':selfie_image}},
                                   Attributes=['ALL'])
response

## Step 4. Authentication 1:n Identification with SearchFaceByImage
-----

For a given input image, first detects the largest face in the image, and then searches the specified collection for matching faces. The operation compares the features of the input face with faces in the specified collection.

<div class="alert alert-info"><strong> Note </strong>
    

- SimilarityThreshold - The minimum level of confidence in the face matches that a match must meet to be included in the FaceMatches array. 

- QualityFilter - A filter that specifies a quality bar for how much filtering is done to identify faces. Filtered faces aren't compared. If you specify AUTO, Amazon Rekognition chooses the quality bar. If you specify LOW, MEDIUM, or HIGH, filtering removes all faces that do not meet the chosen quality bar. 

</div>


In [ ]:
response = rek_client.search_faces_by_image(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    MaxFaces=1,
    FaceMatchThreshold=90,
    QualityFilter='AUTO'
)
response

external_image_id = response["FaceMatches"][0]["Face"]["ExternalImageId"]
similarity_score =  response["FaceMatches"][0]["Similarity"]
face_id = response["FaceMatches"][0]["Face"]["FaceId"]

print("-- FaceID          : {} ".format(face_id))
print("-- Similarity      : {} ".format(similarity_score))
print("-- ExternalImageID : {} \n".format(external_image_id))

print(json.dumps(response, indent=3))  

    

## 1:n images in a collection?

what happens when we have more than one image of a user in a collection? lets check it out! 

<div class="alert alert-info"><strong> Steps </strong>
    

- Here we are going to index several images into our collection 
- then search the collection with a new selfie 
</div>

In [ ]:
images_to_index = ["face_52_0.jpeg", "face_52_1.jpeg", "face_52_2.jpeg"]
external_image_id = "Entity_X_Cannon"

for image_name in images_to_index:
    
    response = rek_client.index_faces(
        CollectionId= collection_name,
        Image={
            'S3Object': {
                'Bucket': bucket_name,
                'Name': image_name,
            }
        },
        ExternalImageId=external_image_id,
        DetectionAttributes=[
            'DEFAULT',
        ],
        MaxFaces=1, # maximum faces detected 
        QualityFilter='AUTO' # apply the quality filter. 
        )
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("ImageName: {}, FaceID: {}".format(image_name, face_id))
    

print("indexing complete")

### Face's Indexed
-----
Now we should have a total of 4 image in the collection for "Entity_X_Cannon". Now the user is going to take a final selfie "face_52_3.jpeg" and we'll search search the index. 

<div class="alert alert-info"><strong> Results </strong>
    

- Check out the similarity of each face found, it should range from 99.99 to 99.96 (Rekognition is extremly accurate) 
- Also note the ExternalImageId of each face, they should all match Entity_X_Cannon
    
</div>

In [ ]:
new_selfie =  "face_52_3.jpeg"

response = rek_client.search_faces_by_image(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': new_selfie,
        }
    },
    MaxFaces=10,
    FaceMatchThreshold=90,
    QualityFilter='AUTO'
)

print("SearchFacesByImage :")
for face in response['FaceMatches']:
    print("-- ExternalImageID : {} \n\tSimilarity : {:.4f} \n\tFaceID     : {}".format(face['Face']['ExternalImageId'],
                                              face['Similarity'],
                                              face['Face']['FaceId']))





## Search within Collection
----- 

What faces matches another face within the collection? well now we have several faces of the same person, lets see what that looks like.

In [ ]:
## -- Snag a FaceID from above --
face_id = " " # enter FaceID from above
try:
    response = rek_client.search_faces(
        CollectionId=collection_name,
        FaceId=face_id,
        MaxFaces=10
    )
    print(response)
except:
    print("--- FaceId not Found ---")


## Clean up
------

As part of our cleanup, we can a delete our collection. This will delete the collection and all the face vectors contained within.

In [ ]:
try:
    rek_client.delete_collection(
        CollectionId=collection_name
    )
except Exception as err:
    print("ERROR: {}".format(err))